In [168]:
import polars as pl
import os

In [169]:
%pwd

'd:\\Repositories\\ml-churn'

### Setting up directory

In [170]:
os.chdir("d://Repositories/ml-churn/")

### Read excel tables

In [171]:
churn_demo  = pl.read_excel("artifacts\data_preprocessing\Telco_customer_churn_demographics.xlsx")
churn_loca  = pl.read_excel("artifacts\data_preprocessing\Telco_customer_churn_location.xlsx")
churn_serv  = pl.read_excel("artifacts\data_preprocessing\Telco_customer_churn_services.xlsx")
churn_stat  = pl.read_excel("artifacts\data_preprocessing\Telco_customer_churn_status.xlsx")

### Join usable tables

In [172]:
new_tab = churn_demo.join(
   churn_loca,how="left", on="Customer ID"
).join(
   churn_serv,how="left", on=["Customer ID","Count"]
).join(
   churn_stat,how="left", on=["Customer ID","Count"]
)

### Inspect column dtypes, and determine usefull columns

In [173]:
new_tab.head()


Customer ID,Count,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Count_right,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Quarter,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Quarter_right,Satisfaction Score,Customer Status,Churn Label,Churn Value,Churn Score,CLTV,Churn Category,Churn Reason
str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,str,f64,f64,str,str,i64,i64,str,str,f64,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,i64,f64,f64,str,i64,str,str,i64,i64,i64,str,str
"""8779-QRDMV""",1,"""Male""",78,"""No""","""Yes""","""No""","""No""",0,1,"""United States""","""California""","""Los Angeles""",90022,"""34.02381, -118…",34.02381,-118.156582,"""Q3""","""No""",0,1,"""None""","""No""",0.0,"""No""","""Yes""","""DSL""",8,"""No""","""No""","""Yes""","""No""","""No""","""Yes""","""No""","""No""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",39.65,39.65,0.0,20,0.0,59.65,"""Q3""",3,"""Churned""","""Yes""",1,91,5433,"""Competitor""","""Competitor off…"
"""7495-OOKFY""",1,"""Female""",74,"""No""","""Yes""","""Yes""","""Yes""",1,1,"""United States""","""California""","""Los Angeles""",90063,"""34.044271, -11…",34.044271,-118.185237,"""Q3""","""Yes""",1,8,"""Offer E""","""Yes""",48.85,"""Yes""","""Yes""","""Fiber Optic""",17,"""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Credit Card""",80.65,633.3,0.0,0,390.8,1024.1,"""Q3""",3,"""Churned""","""Yes""",1,69,5302,"""Competitor""","""Competitor mad…"
"""1658-BYGOY""",1,"""Male""",71,"""No""","""Yes""","""No""","""Yes""",3,1,"""United States""","""California""","""Los Angeles""",90065,"""34.108833, -11…",34.108833,-118.229715,"""Q3""","""No""",0,18,"""Offer D""","""Yes""",11.33,"""Yes""","""Yes""","""Fiber Optic""",52,"""No""","""No""","""No""","""No""","""Yes""","""Yes""","""Yes""","""Yes""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",95.45,1752.55,45.61,0,203.94,1910.88,"""Q3""",2,"""Churned""","""Yes""",1,81,3179,"""Competitor""","""Competitor mad…"
"""4598-XLKNJ""",1,"""Female""",78,"""No""","""Yes""","""Yes""","""Yes""",1,1,"""United States""","""California""","""Inglewood""",90303,"""33.936291, -11…",33.936291,-118.332639,"""Q3""","""Yes""",1,25,"""Offer C""","""Yes""",19.76,"""No""","""Yes""","""Fiber Optic""",12,"""No""","""Yes""","""Yes""","""No""","""Yes""","""Yes""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",98.5,2514.5,13.43,0,494.0,2995.07,"""Q3""",2,"""Churned""","""Yes""",1,88,5337,"""Dissatisfactio…","""Limited range …"
"""4846-WHAFZ""",1,"""Female""",80,"""No""","""Yes""","""Yes""","""Yes""",1,1,"""United States""","""California""","""Whittier""",90602,"""33.972119, -11…",33.972119,-118.020188,"""Q3""","""Yes""",1,37,"""Offer C""","""Yes""",6.33,"""Yes""","""Yes""","""Fiber Optic""",14,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",76.5,2868.15,0.0,0,234.21,3102.36,"""Q3""",2,"""Churned""","""Yes""",1,67,2793,"""Price""","""Extra data cha…"


In [174]:

# Repeated column or not usable information:
# Count, Count_right, Lat Long (already in as separate variables), Quarter_right, 
# Customer Status, Churn Label, Churn Score (used lated for comparison),
# Churn Reason (drop because Churn category already in) 
drop_variables = ["Count","Count_right","Lat Long",
                  "Quarter_right","Customer Status","Churn Label",
                  "Churn Reason","Country", "Referred a Friend","Quarter"]
drop_variables_age = ["Under 30","Senior Citizen"]

usable_tab = new_tab.drop(drop_variables)
usable_tab_no_age = usable_tab.drop(drop_variables_age)

### Determine the null value counts

In [175]:
usable_tab_no_age.null_count()

Customer ID,Gender,Age,Married,Dependents,Number of Dependents,State,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Churn Value,Churn Score,CLTV,Churn Category
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5174


Fill Offer Category None

In [176]:
usable_tab_no_age = usable_tab_no_age.with_columns(
    pl.col("Offer").replace("None", "No Offer")
)

Just Churn Category have nulls, those nulls represent customer that did not churn, so they're filled with a new category "Not Churn"

In [177]:
usable_tab_no_age = usable_tab_no_age.fill_null(value="Not Churn")

Inspect different measures as mean and std from clients

In [178]:
usable_tab_no_age.describe()

statistic,Customer ID,Gender,Age,Married,Dependents,Number of Dependents,State,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Churn Value,Churn Score,CLTV,Churn Category
str,str,str,f64,str,str,f64,str,str,f64,f64,f64,f64,f64,str,str,f64,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""","""7043""","""7043""",7043.0,"""7043""","""7043""",7043.0,"""7043""","""7043""",7043.0,7043.0,7043.0,7043.0,7043.0,"""7043""","""7043""",7043.0,"""7043""","""7043""","""7043""",7043.0,"""7043""","""7043""","""7043""","""7043""","""7043""","""7043""","""7043""","""7043""","""7043""","""7043""","""7043""",7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,"""7043"""
"""null_count""","""0""","""0""",0.0,"""0""","""0""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""",0.0,"""0""","""0""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",null,null,46.509726,null,null,0.468692,null,null,93486.070567,36.197455,-119.756684,1.951867,32.386767,null,null,22.958954,null,null,null,20.515405,null,null,null,null,null,null,null,null,null,null,null,64.761692,2280.381264,1.962182,6.860713,749.099262,3034.379056,3.244924,0.26537,58.50504,4400.295755,null
"""std""",null,null,16.750352,null,null,0.962802,null,null,1856.767505,2.468929,2.154425,3.001199,24.542061,null,null,15.448113,null,null,null,20.41894,null,null,null,null,null,null,null,null,null,null,null,30.090047,2266.220462,7.902614,25.104978,846.660055,2865.204542,1.201657,0.441561,21.170031,1183.057152,null
"""min""","""0002-ORFBO""","""Female""",19.0,"""No""","""No""",0.0,"""California""","""Acampo""",90001.0,32.555828,-124.301372,0.0,1.0,"""No Offer""","""No""",0.0,"""No""","""No""","""Cable""",0.0,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Month-to-Month…","""No""","""Bank Withdrawa…",18.25,18.8,0.0,0.0,0.0,21.36,1.0,0.0,5.0,2003.0,"""Attitude"""
"""25%""",null,null,32.0,null,null,0.0,null,null,92101.0,33.990646,-121.78809,0.0,9.0,null,null,9.21,null,null,null,3.0,null,null,null,null,null,null,null,null,null,null,null,35.5,400.3,0.0,0.0,70.59,605.75,3.0,0.0,40.0,3469.0,null
"""50%""",null,null,46.0,null,null,0.0,null,null,93518.0,36.205465,-119.595293,0.0,29.0,null,null,22.89,null,null,null,17.0,null,null,null,null,null,null,null,null,null,null,null,70.35,1394.55,0.0,0.0,401.44,2108.64,3.0,0.0,61.0,4527.0,null
"""75%""",null,null,60.0,null,null,0.0,null,null,95329.0,38.161321,-117.969795,3.0,55.0,null,null,36.4,null,null,null,27.0,null,null,null,null,null,null,null,null,null,null,null,89.85,3789.2,0.0,0.0,1191.5,4801.93,4.0,1.0,76.0,5381.0,null
"""max""","""9995-HOTOH""","""Male""",80.0,"""Yes""","""Yes""",9.0,"""California""","""Zenia""",96150.0,41.962127,-114.192901,11.0,72.0,"""Offer E""","""Yes""",49.99,"""Yes""","""Yes""","""None""",85.0,"""Yes""","""Yes""","""Yes""","""Yes""","""Yes""","""Yes""","""Yes""","""Yes""","""Two Year""","""Yes""","""Mailed Check""",118.75,8684.8,49.79,150.0,3564.72,11979.34,5.0,1.0,96.0,6500.0,"""Price"""


Merge Columns with same meaning in another Category, in this case Phone service and Multiple lines

In [179]:
def join_columns_phone(a, b):
    if a == 'Yes' and b == 'Yes':
        return a
    elif a == 'Yes' and b == 'No':
        return b
    elif a == 'No' and b == 'No':
        return 'No Service'
        
def join_columns_internet(a,b):
    if a == 'No' and b == 'None':
        return 'No Service'
    else:
        return b
    
usable_tab_no_age = usable_tab_no_age.with_columns(pl.struct(['Phone Service','Multiple Lines']) \
       .map_elements(lambda x: join_columns_phone(x['Phone Service'], x['Multiple Lines'])).alias('Multiple Lines'))

usable_tab_no_age.drop_in_place("Phone Service")

Phone Service
str
"""No"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""No"""
"""Yes"""


Merge Internet service as a category of Internet Type

In [180]:
def join_columns_internet(a,b):
    if a == 'No' and b == "None":
        return 'No Service'
    else:
        return b
    
usable_tab_no_age = usable_tab_no_age.with_columns(pl.struct(['Internet Service','Internet Type']) \
    .map_elements(lambda x: join_columns_internet(x['Internet Service'], x['Internet Type'])).alias('Internet Type'))

usable_tab_no_age.drop_in_place("Internet Service")

Internet Service
str
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""
"""Yes"""


In [181]:
def join_columns_all(a,b):
    if a == "No Service" and b == "No":
        return "No Service"
    else:
        return b
    
cols = ['Online Security', 'Online Backup',
       'Device Protection Plan', 'Premium Tech Support', 'Streaming TV',
       'Streaming Movies', 'Streaming Music', 'Unlimited Data']

for col in cols:

    usable_tab_no_age = usable_tab_no_age.with_columns(pl.struct(['Internet Type',col]) \
        .map_elements(lambda x: join_columns_all(x['Internet Type'], x[col])).alias(col))

    # usable_tab_no_age.drop_in_place(col)

usable_tab_no_age


Customer ID,Gender,Age,Married,Dependents,Number of Dependents,State,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Avg Monthly Long Distance Charges,Multiple Lines,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Churn Value,Churn Score,CLTV,Churn Category
str,str,i64,str,str,i64,str,str,i64,f64,f64,i64,i64,str,f64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,i64,f64,f64,i64,i64,i64,i64,str
"""8779-QRDMV""","""Male""",78,"""No""","""No""",0,"""California""","""Los Angeles""",90022,34.02381,-118.156582,0,1,"""No Offer""",0.0,"""No Service""","""DSL""",8,"""No""","""No""","""Yes""","""No""","""No""","""Yes""","""No""","""No""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",39.65,39.65,0.0,20,0.0,59.65,3,1,91,5433,"""Competitor"""
"""7495-OOKFY""","""Female""",74,"""Yes""","""Yes""",1,"""California""","""Los Angeles""",90063,34.044271,-118.185237,1,8,"""Offer E""",48.85,"""Yes""","""Fiber Optic""",17,"""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Credit Card""",80.65,633.3,0.0,0,390.8,1024.1,3,1,69,5302,"""Competitor"""
"""1658-BYGOY""","""Male""",71,"""No""","""Yes""",3,"""California""","""Los Angeles""",90065,34.108833,-118.229715,0,18,"""Offer D""",11.33,"""Yes""","""Fiber Optic""",52,"""No""","""No""","""No""","""No""","""Yes""","""Yes""","""Yes""","""Yes""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",95.45,1752.55,45.61,0,203.94,1910.88,2,1,81,3179,"""Competitor"""
"""4598-XLKNJ""","""Female""",78,"""Yes""","""Yes""",1,"""California""","""Inglewood""",90303,33.936291,-118.332639,1,25,"""Offer C""",19.76,"""No""","""Fiber Optic""",12,"""No""","""Yes""","""Yes""","""No""","""Yes""","""Yes""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",98.5,2514.5,13.43,0,494.0,2995.07,2,1,88,5337,"""Dissatisfactio…"
"""4846-WHAFZ""","""Female""",80,"""Yes""","""Yes""",1,"""California""","""Whittier""",90602,33.972119,-118.020188,1,37,"""Offer C""",6.33,"""Yes""","""Fiber Optic""",14,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",76.5,2868.15,0.0,0,234.21,3102.36,2,1,67,2793,"""Price"""
"""4412-YLTKF""","""Female""",72,"""No""","""Yes""",1,"""California""","""Pico Rivera""",90660,33.989524,-118.089299,0,27,"""Offer C""",3.33,"""Yes""","""Fiber Optic""",18,"""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",78.05,2135.5,0.0,10,89.91,2235.41,1,1,95,4638,"""Competitor"""
"""0390-DCFDQ""","""Female""",76,"""Yes""","""Yes""",2,"""California""","""Los Alamitos""",90720,33.79499,-118.065591,1,1,"""Offer E""",15.28,"""No""","""Fiber Optic""",30,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Mailed Check""",70.45,70.45,0.0,0,15.28,85.73,2,1,76,3964,"""Other"""
"""3445-HXXGF""","""Male""",66,"""Yes""","""No""",0,"""California""","""Sierra Madre""",91024,34.168686,-118.057505,6,58,"""Offer B""",0.0,"""No Service""","""DSL""",24,"""No""","""Yes""","""Yes""","""No""","""No""","""Yes""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Bank Withdrawa…",45.3,2651.2,40.95,0,0.0,2610.25,1,1,91,5444,"""Dissatisfactio…"
"""2656-FMOKZ""","""Female""",70,"""No""","""Yes""",2,"""California""","""Pasadena""",91106,34.139402,-118.128658,0,15,"""Offer D""",44.07,"""Yes""","""Fiber Optic""",19,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""Month-to-Month…","""Yes""","""Mailed Check""",74.45,1145.7,0.0,0,661.05,1806.75,2,1,91,5717,"""Dissatisfactio…"


Write the data

In [182]:
usable_tab_no_age.write_csv("artifacts/data_preprocessing/usable_data.csv")